## Importação das bibliotecas e funções

In [1]:
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import os
import datetime
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

In [2]:
# Usando Google Colab:
'''
from google.colab import drive
drive.mount('/content/drive')

pasta = str("/content/drive/MyDrive/Python/TCC-Cloud/Data/")
print(pasta)

preços = pd.read_csv(pasta + "Preços.csv",index_col='Data',parse_dates=True)
semestres = pd.read_csv(pasta + "Semestres.csv",parse_dates=True)

Datas = pd.DataFrame(preços.index)
Datas['indice'] = Datas.index
log_data = np.log(preços)
'''

'\nfrom google.colab import drive\ndrive.mount(\'/content/drive\')\n\npasta = str("/content/drive/MyDrive/Python/TCC-Cloud/Data/")\nprint(pasta)\n\npreços = pd.read_csv(pasta + "Preços.csv",index_col=\'Data\',parse_dates=True)\nsemestres = pd.read_csv(pasta + "Semestres.csv",parse_dates=True)\n\nDatas = pd.DataFrame(preços.index)\nDatas[\'indice\'] = Datas.index\nlog_data = np.log(preços)\n'

In [3]:
# Especificando o caminho padrão do projeto no computador:
pasta = str(Path.cwd().parents[0])
print(pasta)

preços = pd.read_csv(pasta + "/Data/Preços.csv",index_col='Data',parse_dates=True)
semestres = pd.read_csv(pasta + "/Data/Semestres.csv",parse_dates=True)

Datas = pd.DataFrame(preços.index)
Datas['indice'] = Datas.index
log_data = np.log(preços)

/mnt/c/Users/MarceloPolecram/Python/MEU_TCC


In [4]:
# MQO para encontrar o coeficiente de cointegração e criando a serie do spread
def OLS(data_ticker1, data_ticker2):
    spread = sm.OLS(data_ticker1,data_ticker2)
    spread = spread.fit()
    return data_ticker1 + (data_ticker2 * -spread.params[0]), spread.params[0]


# ADF test
def ADF(spread):
    return ts.adfuller(spread) # H0: Raiz unitária.


# Encontra o coeficiente de cointegração e realiza o ADF test
def ADF_test(data_ticker1, data_ticker2):
    ols = OLS(data_ticker1, data_ticker2)
    spread = ols[0]
    gamma = ols[1]
    return ADF(spread),gamma


# Encontra os pares cointegrados
def find_cointegrated_pairs_mod(data):
    try:
        print(f"Finding cointegrated pairs for shape {np.shape(data)}")
        n = data.shape[1]
        pvalue_matrix = np.ones((n, n))
        gammas_matrix = np.ones((n, n))
        keys = data.keys()
        for i in range(n):
            for j in range(i+1, n):
                S1 = keys[i]
                S2 = keys[j]
                #print(f"Testing cointegration for pairs {S1} and {S2}")
                result = ADF_test(data[S1], data[S2])
                gammas_matrix[i, j] =result[1] # gamma
                pvalue = result[0][1] # pvalue
                pvalue_matrix[i, j] = pvalue
            if(i%50 == 0):
                print(f"Finished pairs for stock {keys[i]}")
        return pvalue_matrix, gammas_matrix
    except Exception as e:
        print(e)


# Ordenando os melhores pares
def top_coint_pairs(data,pvalue_matrix,gamma, alpha,semestre,n):
#alpha = nivel de significancia para o teste ADF
#n = top n ativos com o menor pvalue
    alpha_filter = np.where(pvalues < alpha)
    pvalues_f = pvalues[alpha_filter] # pvalores menores que alpha
    #print(f"Alpha filter rows len {len(alpha_filter[0])} | cols len {len(alpha_filter[1])} | value {alpha_filter}")
    #print(f"pvalues_f len: {np.shape(pvalues_f)} | value: {pvalues_f}")
    stock_a = data.columns[list(alpha_filter)[0]] # relacionando o pvalor com a ação A
    stock_b = data.columns[list(alpha_filter)[1]] # relacionando o pvalor com a ação B
    gammas_f = gammas[alpha_filter] # relacionando o pvalor com o gamma
    N = len(list(alpha_filter[0])) # quantidade de pares cointegrados

    d = []
    for i in range(N):
        pair_dict = {
            'Stock A': stock_a[i],
            'Stock B': stock_b[i],
            'P-Values': pvalues_f[i],
            'Gamma': gammas_f[i],
            'Semestre' : semestre
        }
        #if(i%2000 == 0):
        #print(f"Appending pair dict: {pair_dict}")
        d.append(pair_dict)

    return pd.DataFrame(d).sort_values(by="P-Values").iloc[:n,]


# Calcula os retornos da carteira e armazenando em um data frame
def calculate_profit(pair, spread, threshold, par1, par2, resumo, semester, gamma):
    #print(f"Calculating profits for pair {par1}-{par2}")

    #print(f"Spread series: {spread}")

    date_format = "%Y-%m-%d"

    log_ret = spread.diff() # log return eh o incremento
    #print(f"log_ret len: {len(log_ret)} | variable: {log_ret}")
    dias = spread.index
    #print(f"Dias len: {len(dias)} | variable: {dias}")
    z_score = (spread-spread.mean())/spread.std()
    #z_score.plot() # Deixa a memória RAM muito sobrecarregada
    #print(f"Z-score index: {z_score.index} | variable: {z_score}")
    portfolio_return = []
    pos = 0 # 0: sem posição aberta
            # 1: Comprei o meu portfolio h = (1,-gamma)
            # -1: Vendi o meu portfolio h = -(1,-gamma)

    dias_abertura = []
    dias_fechamento = []

    count = 0
    dia_abertura = 0
    dia_fechamento = 0

    closing_threshold = 0.0

    for i in range(1, len(z_score)):

        if (z_score.iloc[i][0] > threshold) and (pos == 0):
            # Posição fechada no par e com sinal short 1st e long 2nd
            pos = -1

            count += 1
            dia_abertura = dias[i] - dias[0]
            retornos_op = []


        elif (z_score.iloc[i][0] < -threshold)  and (pos == 0):
            # Posição fechada no par e com sinal de long 1st e short 2nd
            pos = 1

            count += 1
            dia_abertura = dias[i] - dias[0]
            retornos_op = []

        else:
            #print(f"Dia {i} | Pos {pos} | log_ret {log_ret[i]} | S1 return {returns[par1][i]} | S2 return {returns[par2][i]} | Net {pos*(returns[par1][i] - gamma*returns[par2][i])}")
            if (pos != 0) and ((dias[i] - dias[0] - dia_abertura) == duration_limit):
                #Fechando operações maiores que 50 dias
                portfolio_return.append(log_ret.iloc[i][0]*pos)
                pos = 0
                dia_fechamento = dias[i] - dias[0]
                delta_dias = dia_fechamento - dia_abertura
                if sum(retornos_op) < stop_loss:
                    print(f"Sum: {sum(retornos_op)}")
                    retorno_op = stop_loss
                    print(f"Retorno_op: {retorno_op}")
                else:
                    retornos_op.append(log_ret.iloc[i][0]*pos)
                    retorno_op = pd.Series(retornos_op).sum()

                resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, True])


            elif (pos == 1) and (z_score.iloc[i][0] >= -closing_threshold or sum(retornos_op) < stop_loss):
                # Posição vendida aberta no par e com sinal de convergência
                portfolio_return.append(log_ret.iloc[i][0]*pos)
                pos = 0

                dia_fechamento = dias[i] - dias[0]
                delta_dias = dia_fechamento - dia_abertura
                if sum(retornos_op) < stop_loss:
                    print(f"Sum: {sum(retornos_op)}")
                    retorno_op = stop_loss
                    print(f"Retorno_op: {retorno_op}")
                else:
                    retornos_op.append(log_ret.iloc[i][0]*pos)
                    retorno_op = pd.Series(retornos_op).sum()

                resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, True])

            elif (pos == -1) and (z_score.iloc[i][0] <= closing_threshold or sum(retornos_op) < stop_loss):
                portfolio_return.append(log_ret.iloc[i][0]*pos)
                pos = 0

                dia_fechamento = dias[i] - dias[0]
                delta_dias = dia_fechamento - dia_abertura
                if sum(retornos_op) < stop_loss:
                    print(f"Sum: {sum(retornos_op)}")
                    retorno_op = stop_loss
                    print(f"Retorno_op: {retorno_op}")
                else:
                    retornos_op.append(log_ret.iloc[i][0]*pos)
                    retorno_op = pd.Series(retornos_op).sum()

                resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, True])

            elif (pos == 1) and (z_score.iloc[i][0] < -closing_threshold):
                # Posição vendidada aberta no par aberta e sem convergência
                portfolio_return.append(log_ret.iloc[i][0]*pos)
                retornos_op.append(log_ret.iloc[i][0]*pos)


            elif (pos == -1) and (z_score.iloc[i][0] > closing_threshold):
                portfolio_return.append(log_ret.iloc[i][0]*pos)
                retornos_op.append(log_ret.iloc[i][0]*pos)

            else:
                # Sem posição aberta e nem sinal de entrada

                if pos != 0:
                    dia_fechamento = dias[i] - dias[0]
                    delta_dias = dia_fechamento - dia_abertura
                    retornos_op.append(log_ret.iloc[i][0]*pos)
                    retorno_op = pd.Series(retornos_op).sum()

                    resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, True])

                pos = 0

    if pos != 0:
        # Operação sem convergência
        pos = 0

        dia_fechamento = dias[i] - dias[0]
        delta_dias = dia_fechamento - dia_abertura
        retorno_op = pd.Series(retornos_op).sum()
        print(f"Par {par1}-{par2} sem convergência, retorno_op: {retorno_op}")
        resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, False])

    #print(f"Total return: {sum(pair_returns)} | Pair returns: {pair_returns}")
    #print(f"Conversão do par: {pos}")
    total_ret = pd.Series(portfolio_return).sum()

    return total_ret, resumo


# Calcula o expoente de hurst
def get_hurst_exponent(time_series):

    # Definindo o intervalo de taus
    max_tau = round(len(time_series)/4)
    taus = range(2, max_tau)

    # Calculando a variável k
    k = [np.std(np.subtract(time_series[tau:], time_series[:-tau])) for tau in taus]

    'To calculate the Hurst exponent, we first calculate the standard deviation of the differences between a series and its lagged version, for a range of possible lags.'

    # Calculate the slope of the log plot -> the Hurst Exponent
    reg = np.polyfit(np.log(taus), np.log(k), 1)

    'We then estimate the Hurst exponent as the slope of the log-log plot of the number of lags versus the mentioned standard deviations.'

    return reg[0]

## Leitura das séries de Preços e retornos das ações,também dos semestres do período total

In [5]:
no_pairs = 5
duration_limit = 50
years = 2023 - 1995
threshold = 2

alpha = 0.05

days, num_assets = np.shape(preços)

stop_loss = float('-inf')


In [6]:
np.shape(preços)

(7563, 212)

## Execução da Estratégia:

In [7]:
results = []
resumos = []

coint_pairs_df = []

inicio = 0

for big_loop in range(0, len(semestres) - 3):
    print(f"Starting period {big_loop} | Past days: {inicio}")

    # Listando os dias dos intervalos
    inicio = (Datas == semestres['Data'][big_loop]).query("Data == True").index[0]
    twelve_months = (Datas == semestres['Data'][big_loop + 2]).query("Data == True").index[0]
    six_months = (Datas == semestres['Data'][big_loop + 3]).query("Data == True").index[0]

    # Limpeza das ações não listadas no período
    ps = log_data.iloc[inicio:twelve_months,:].dropna(how="any",axis=1) # Só usar se der erro
    listed = ps.iloc[0]
    listed_num = np.sum(listed.value_counts())
    print(f"Listed stocks for the period: {listed_num}")

    log_data_filtrado = log_data.columns.isin(listed.index)
    listed_indexes = np.where(log_data_filtrado > 0)[0]
    listed_stocks = listed.index

    # Achando os pares cointegrados
    #ps = log_data.iloc[inicio:twelve_months,listed_indexes]
    #ps = log_data.iloc[inicio:twelve_months,listed_indexes].dropna(how="any",axis=1) # Só usar se der erro
    pvalues, gammas = find_cointegrated_pairs_mod(ps)
    print(pvalues.shape,gammas.shape)
    cp = top_coint_pairs(ps, pvalues, gammas, alpha, big_loop,no_pairs)
    coint_pairs_df.append(cp)
    #print(coint_pairs_df)

Starting period 0 | Past days: 0
Listed stocks for the period: 57
Finding cointegrated pairs for shape (260, 57)
Finished pairs for stock VALE3
Finished pairs for stock SUZB5
(57, 57) (57, 57)
Starting period 1 | Past days: 129
Listed stocks for the period: 60
Finding cointegrated pairs for shape (262, 60)
Finished pairs for stock VALE3
Finished pairs for stock SHAP4
(60, 60) (60, 60)
Starting period 2 | Past days: 259
Listed stocks for the period: 60
Finding cointegrated pairs for shape (261, 60)
Finished pairs for stock VALE3
Finished pairs for stock SHAP4
(60, 60) (60, 60)
Starting period 3 | Past days: 389
Listed stocks for the period: 63
Finding cointegrated pairs for shape (261, 63)
Finished pairs for stock VALE3
Finished pairs for stock PMAM4
(63, 63) (63, 63)
Starting period 4 | Past days: 521
Listed stocks for the period: 62
Finding cointegrated pairs for shape (261, 62)
Finished pairs for stock VALE3
Finished pairs for stock PALF3
(62, 62) (62, 62)
Starting period 5 | Past da

In [8]:
# Vendo o Lucro da estratégia
inicio = 0

for big_loop in range(0, len(semestres) - 3):
    print(f"Starting period {big_loop} | Past days: {inicio}")

    coint_pairs_sem_df = coint_pairs_df[big_loop]

    # Listando os dias dos intervalos
    inicio = (Datas == semestres['Data'][big_loop]).query("Data == True").index[0]
    twelve_months = (Datas == semestres['Data'][big_loop + 2]).query("Data == True").index[0]
    six_months = (Datas == semestres['Data'][big_loop + 3]).query("Data == True").index[0]

    resultado = []
    for i in range(0,coint_pairs_sem_df.shape[0]):
        S1_name = coint_pairs_sem_df.iloc[i, 0]
        S2_name = coint_pairs_sem_df.iloc[i, 1]
        gamma_1_2 = coint_pairs_sem_df.iloc[i, 3]

        #Aqui tá o problema das datas:
        S1 = log_data[S1_name].iloc[twelve_months:six_months] # periodo de teste
        S2 = log_data[S2_name].iloc[twelve_months:six_months] # periodo de teste

        #spread, convertendo Datas para nºs de dias
        spread_ = S1 - gamma_1_2*S2
        spread_.name = "spread"
        spread = pd.merge(spread_,Datas, on="Data")
        spread.index = spread['indice']
        spread.drop(['Data','indice'],inplace=True,axis=1)

        #print(spread)
        # Pegando o resultado da estratégia
        #resultado.append(np.exp(calculate_profit(spread,1.65)))
        ret, resumos = calculate_profit(i, spread, threshold, S1_name, S2_name, resumos, big_loop, gamma_1_2)
    print("-------------------------------------------------")

Starting period 0 | Past days: 0
Par ITUB4-CMIG4 sem convergência, retorno_op: 0.15779485214685773
Par ITUB4-BBDC4 sem convergência, retorno_op: 0.01114079541008528
Par VIVT3-PTIP4 sem convergência, retorno_op: -0.05136986627720619
-------------------------------------------------
Starting period 1 | Past days: 129
-------------------------------------------------
Starting period 2 | Past days: 259
Par BRKM5-CMIG3 sem convergência, retorno_op: 0.48705470538863604
Par VALE3-BRKM5 sem convergência, retorno_op: 1.2954964658416694
Par CSNA3-BRKM5 sem convergência, retorno_op: 3.713265131797807
Par BBAS4-UNIP6 sem convergência, retorno_op: -0.12037370909618605
Par BRKM5-SHAP4 sem convergência, retorno_op: 0.43865676725847724
-------------------------------------------------
Starting period 3 | Past days: 389
Par ABEV3-CMIG3 sem convergência, retorno_op: 0.14902584505946437
Par CMIG3-BRDT4-old sem convergência, retorno_op: 0.03589010473587262
Par BBAS4-CMIG3 sem convergência, retorno_op: 0.1

## Salvando os resultados

In [9]:
cols = ['Operação', 'Semestre', 'Abertura', 'Fechamento', 'Dias', 'Retorno total', 'Ticker 1', 'Ticker 2', 'Converged']
df = pd.DataFrame(resumos, columns = cols)
df['Index'] = df['Ticker 1'].astype(str) + '-' + df['Ticker 2'].astype(str) + '-' + df['Operação'].astype(str)
df['Retorno total - exp'] = np.exp(df['Retorno total'])

#Drive
'''
pasta2 = str("/content/drive/MyDrive/Python/TCC-Cloud/Distancia-Cointegração/result_cointegracao/")
print(pasta2)
'''

#PC:
pasta2 = str(pasta + "/Distancia-Cointegração/result_cointegracao/")
print(pasta2)

os.makedirs(pasta2, exist_ok=True)
df.to_csv(pasta2 + "operations_C5.csv", sep=',', index=False)

/mnt/c/Users/MarceloPolecram/Python/MEU_TCC/Distancia-Cointegração/result_cointegracao/


# Rascunhos: